## imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# API functions

In [ ]:
import os
import requests
from dotenv import load_dotenv, dotenv_values
load_dotenv()

def getToken():
    url = "https://accounts.spotify.com/api/token"
    H = {"Content-type": "application/x-www-form-urlencoded"}
    d = {
        "grant_type": "client_credentials",
        "client_id": os.getenv("CLIENT_ID"),
        "client_secret": os.getenv("CLIENT_SECRET")
    }
    res = requests.post(url, headers=H, data=d)
    
    # if error/not working
    if res.status_code != 200:
        Exception("getToken: ",res.text)
    
    # example output 'BQCq8oiJHxi.....iv_5sAuoF1YwhQADyc'
    return res.json()['access_token']

# get audio features using track id
def getAF(track_id):
    url = "https://api.spotify.com/v1/audio-features/"+track_id
    H = {"Authorization": "Bearer " + getToken() }
    res = requests.get(url, headers=H)
    
    if res.status_code != 200:
        Exception("getAF: ",res.text)
    
    return res.json()

# getAF("7fvUMiyapMsRRxr07cU8Ef")


## import file

In [ ]:
path = './datapack/spotify_songs.csv'
df = pd.read_csv(path)

## clean data

In [3]:
# drop any rows with missing value from any column, 
# axis 0 to drop the rows not the columns

df = df.dropna(how='any', axis=0)

## process

### Visualization

In [ ]:
audio_features = ['track_popularity', 'danceability','energy', 
'key', 'loudness', 'mode',
'speechiness','acousticness','instrumentalness',
'liveness', 'valence', 'tempo']

correlation = df[audio_features].corr()
plt.figure(figsize=(10,9))
sns.heatmap(correlation, annot=True)
plt.title("Correlation Heatmap")
plt.show()